In [1]:
'''匯入套件'''

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 隨機產生
import random

# 正規表達式
import re

# 數學
import math

# 系統資訊
import sys, traceback, os




'''全域變數初始化'''

# 自定義搜尋語法(一定要在實際的 Web of Science 網站執行過，確定檢索過程語法無誤、核心合輯結果數跟預期一樣)
arrQueryString = [
    "TI=meta-analysis",
    "TI=evidence review",
    "TI=systematic review",
    "SO=Cochrane Database of Systematic Reviews",
    "AK=meta-analysis",
    "AK=evidence review",
    "AK=systematic review",
    "KP=meta-analysis",
    "KP=evidence review",
    "KP=systematic review",
    "#1 OR #2 OR #3 OR #4 OR #5 OR #6 OR #7 OR #8 OR #9 OR #10",
    "PY=2010-2020",
    "#11 AND #12"
]

# 文件類型 (留空代表不用選擇文件類型)
strDocType = ''

# 隨機睡眠最大時間
sleepMaxSeconds = 3 

# 隨機睡眠最小時間
sleepMinSeconds = 2 

# [ [1, 500], [501, 1000], ... ] 等下載頁數範圍
arrRecords = [] 

# 下載資料夾 (在 unix-like 環境要用 / 這個斜線)
download_path = os.getcwd() + "\\" + "wos_records" 

# chrome 執行檔路徑 (在 unix-like 環境要用 / 這個斜線)
executable_path = os.getcwd() + "\\" + "chromedriver.exe" 

# screenshot 的路徑
screenshot_path = os.getcwd() + "\\" + "screenshot.png"

# Web Of Science 頁面連結
urlWoS = "http://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch"




'''自訂函式'''

# 回傳隨機秒數，協助元素等待機制
def _sleepRandomSeconds():
    sleep( random.randint(sleepMinSeconds, sleepMaxSeconds) )

# 初始化設定
def init():
    #新增預設下載目錄，如果不存在，就新增資料夾
    if not os.path.exists(download_path):
        os.makedirs(download_path, exist_ok = True)

# 點選「進階檢索」連結
def _goToAdvancedSearchPage():
    #前往 Web of Science 頁面
    driver.get(urlWoS)
    
    #確認即將點選的「進階檢索」連結，所放置的網頁元素在不在
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "ul.searchtype-nav")
        )
    )
    
    #按下「進階檢索」
    li = driver.find_elements(By.CSS_SELECTOR, "ul.searchtype-nav li")
    li[3].find_element(By.CSS_SELECTOR, 'a').click()

# 整理搜尋用的字串
def _setFilterCondition(strQuery):
    #確認 textarea 元素是否存在
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.AdvSearchBox textarea")
        )
    )
    
    #清空 textarea
    driver.find_element(By.CSS_SELECTOR, "div.AdvSearchBox textarea").clear()
    
    #將檢索語法的字串，放到 textarea 當中
    driver.find_element(By.CSS_SELECTOR, "div.AdvSearchBox textarea").send_keys(strQuery)

    #休閒一段時間 (依情況複製多次)
    _sleepRandomSeconds()
        
#選擇文件類型
def _setDocumentType(_type):
    # 讓文件類型指定 type 的字串值來選擇 select option 元素
    js_script = '''(
        function () {{
            let option = document.querySelector('select[name="value(input3)"] option[value="{}"]');
            option.setAttribute('selected', true);
        }}
    )();
    '''.format(_type)
    
    #執行 javsascript 語法
    driver.execute_script(js_script)
        
# 清除編號的歷史記錄
def _clearSerialNumberHistory():
    #前往 WoS「進階檢索」連結的頁面
    _goToAdvancedSearchPage()

    #等待「進階搜尋」的連結出現，再按下該連結
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.AdvSearchBox textarea")
        )
    )
    
    #頁面快照
    _screenshot()

    #清空 textarea (輸入檢索語法的那個文字欄位)
    driver.find_element(By.CSS_SELECTOR, "div.AdvSearchBox textarea").clear()
    
    #按下 History 表格右側「刪除檢索集」下面的「全選」
    driver.find_element(By.CSS_SELECTOR, "button#selectallTop").click()
    
    #按下 History 表格右側「刪除檢索集」下面的「刪除」
    driver.find_element(By.CSS_SELECTOR, "button#deleteTop").click()

# 前往檢索結果的連結
def _goToSearchResultPage():
    #判斷查詢後歷史記錄編號，是否低於最高檢索數
    objIndexNum = driver.find_elements(By.CSS_SELECTOR, "table tbody tr[id^='set_'] td div.historyResults")
    
    #取得當前檢索結果的小計
    div = driver.find_element(By.CSS_SELECTOR, "table tbody tr[id^='set_" + str(len(objIndexNum)) + "'] td div.historyResults")
    numResult = div.text
    numResult = int(re.sub(r"\s|,", "", numResult))
    
    #檢索小計除以500，例如 2819 / 500 = 5.638
    _set = math.floor(numResult / 500)
    
    #組數除完的餘數，用來加在最後一組
    _remainder = numResult % 500
    
    #建立例如 [ [1,500], [501, 1000], [1001, 1500], [1501, 2000], [2001, 2500] , [2501, 2819] ]
    for i in range(0, _set + 1):
        if i == _set:
            arrRecords.append([i * 500 + 1, (i * 500) + _remainder])
        else:
            arrRecords.append([i * 500 + 1, (i + 1) * 500])
    
    print(f"Web Of Science 核心合輯結果數: {numResult}")
    
    print("數量切割結果(每 500 筆一組):")
    print(arrRecords)
    
    #按下檢索小計的連結，例如 #6
    div.find_element(By.CSS_SELECTOR, "a").click()
        
# 下載期刊資訊
def _downloadJournalPlaneTextFile():
    #用來確認是否按過 Export 鈕（之後會變成其它按鈕元素，所以會用 More 來按）
    firstExportFlag = False
    
    #按下匯出鈕，跳出選單(網頁上有重複的元素，例如表格頭尾都有匯出鈕，這裡選擇一個來按)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "button#exportTypeName")
        )
    )
    buttonsExport = driver.find_elements(By.CSS_SELECTOR, "button#exportTypeName")
    buttonsExport[0].click()

    #走訪選單連結文字，找到合適字串，就點按該連結，並跳出迴圈
    li_elms = driver.find_elements(By.CSS_SELECTOR, "ul#saveToMenu li.subnav-item")
    for li in li_elms:
        listSubString = ['Other File Formats', '其他檔案格式']
        subString = li.find_element(By.CSS_SELECTOR, "a").get_attribute('innerText')
        if any( subString in s for s in listSubString ):
            li.find_element(By.CSS_SELECTOR, "a").click()
            break

    for i in range(0, len(arrRecords)):
        #確認是否執行第一次匯出，已匯出，之後都按 More 按鈕
        if firstExportFlag == True:
            #按下 More 鈕，跳出選單(網頁上有重複的元素，例如表格頭尾都有匯出鈕，這裡選擇一個來按)
            buttonsExportMore = driver.find_elements(By.CSS_SELECTOR, "button#exportMoreOptions")
            buttonsExportMore[0].click()

            #走訪選單連結文字，找到合適字串，就點按該連結，並跳出迴圈
            li_elms = driver.find_elements(By.CSS_SELECTOR, "ul#saveToMenu li.subnav-item")
            for li in li_elms:
                listSubString = ['Other File Formats', '其他檔案格式']
                subString = li.find_element(By.CSS_SELECTOR, "a").get_attribute('innerText')
                if any( subString in s for s in listSubString ):
                    li.find_element(By.CSS_SELECTOR, "a").click()
                    break
        
        #選擇匯出檔案的資料筆數(Records)，一次不能超過 500 筆
        driver.find_element(By.CSS_SELECTOR, 'input#numberOfRecordsRange[type="radio"]').click()
        driver.find_element(By.CSS_SELECTOR, 'input#markFrom[type="text"]').clear()
        driver.find_element(By.CSS_SELECTOR, 'input#markFrom[type="text"]').send_keys(arrRecords[i][0])
        driver.find_element(By.CSS_SELECTOR, 'input#markTo[type="text"]').clear()
        driver.find_element(By.CSS_SELECTOR, 'input#markTo[type="text"]').send_keys(arrRecords[i][1])

        #選擇「記錄內容」
        selectBibFieldsOptions = driver.find_elements(By.CSS_SELECTOR, "select#bib_fields option")
        selectBibFieldsOptions[3].click()

        #選擇「檔案格式」
        selectSaveOptions = driver.find_elements(By.CSS_SELECTOR, "select#saveOptions option")
        selectSaveOptions[3].click()

        #按下匯出鈕，此時等待下載，直到開始下載，才會往程式下一行執行
        driver.find_element(By.CSS_SELECTOR, "button#exportButton").click()
        
        #休閒一段時間 (依情況複製多次，網路不穩或速度慢，影響下載，可以多複製幾個)
        _sleepRandomSeconds()
        _sleepRandomSeconds()
        
        #第一次匯出已完成，之後不按 Export 鈕按，改為 More 按鈕（按下後，可以選擇 Other File Formats）
        firstExportFlag = True
        

#畫面快照
def _screenshot():    
    #對 History 表格元素進行快照
    elms_table = driver.find_elements(By.CSS_SELECTOR, 'table[width="100%"]')
    elms_table[8].screenshot(screenshot_path)
    
# 主程式
def main():
    #點選「進階檢索」連結
    _goToAdvancedSearchPage()

    #迭代自訂期刊
    for i in range(0, len(arrQueryString)):
        #整理搜尋用的字串
        _setFilterCondition(arrQueryString[i])

        #選擇文件類型（目前設定 Article）
        if strDocType != '':
            _setDocumentType('Article')

        #按下搜尋
        driver.find_element(By.CSS_SELECTOR, "button#search-button").click()

        #若是查詢到最後，產生新的 #編號，則按下 #編號旁邊的數字超連結，進入下載階段
        if (i + 1) == len(arrQueryString):
            '''
            查詢 len(arrQueryString) 個語法，並取得最後查詢的期刊總數，再進入期刊總數的連結
            '''
            #前往檢索結果的連結
            _goToSearchResultPage()

            #迭代下載資料(這邊會執行比較久)
            _downloadJournalPlaneTextFile()

            #刪除搜尋的歷史記錄
            _clearSerialNumberHistory()
                
#關閉 chrome
def close():
    driver.quit()

In [ ]:
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器，在背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截
options.add_argument("--disable-notifications")   #取消通知
options.add_experimental_option("prefs", {
  "download.default_directory": download_path #預設下載路徑
})

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( 
    options = options, #啟動瀏覽器工具的選項
    executable_path = executable_path #chrome driver 主程式放置的路徑
)

# 主程式
if __name__ == '__main__':
    init()
    main()
    close()